In [534]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import numpy as np
import scipy.stats

from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
import time
import pytz

import ipywidgets


### DECLARACAO DE FUNCOES ###

In [657]:

# Baixa base de dados e retorna uma base da forma [data,close_ativo_1, close_ativo_2]
def baixar_yahoo(tickers,inicio,fim,baixayahoo):

    # Percorre a lista tickers, lendo o yahoo e salvando no DataFrame db
    db = pd.DataFrame()
    if (baixayahoo == True):
        
        for i in tickers:
            cot = pdr.DataReader(i, 'yahoo', inicio,fim)
            cot['Ativo']=i
            db = pd.concat([db,cot],sort=True)  

        db.to_csv('Ativos.csv')
        #db.to_excel('ativos.xlsx')
    
    db = pd.read_csv('Ativos.csv',parse_dates=[0], infer_datetime_format = True)
    #db = pd.read_excel('ITSA4GOLL4.xlsx')
    db = db[['Ativo','Date','Open','High','Low','Adj Close']].rename(columns={'Adj Close': 'Close'})    
    
    # Cria Tabela com os fechamentos dos dois ativos [DATA, Close1, Close2]
    esquerda = db[db['Ativo']==tickers[0]].set_index('Date')['Close']
    direita = db[db['Ativo']==tickers[1]].set_index('Date')['Close']
    par = pd.merge(left=esquerda, right=direita, left_on='Date', right_on='Date').rename(columns={'Close_x': tickers[0], 'Close_y': tickers[1]})

    #par.to_excel('Par.xlsx')
    
    return par

# Baixa base de dados e retorna uma base da forma [data,close_ativo_1, close_ativo_2]
def Mt5_dados(tickers,inicio,fim):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")

    # obtendo o par de acoes com base nas datas enviadas nas strings, definindo o timezone para nao interfir com o tisp do mt5
    stock1 = mt5.copy_rates_range(tickers[0], mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),datetime(fim[0],fim[1],fim[2]))
    stock2 = mt5.copy_rates_range(tickers[1], mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),datetime(fim[0],fim[1],fim[2]))

    # concluímos a conexão ao MetaTrader 5
    #mt5.shutdown()

    esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
    direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})

    stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
    stock_df.drop('time',axis=1,inplace = True)

    return stock_df.set_index('Date')

In [759]:
def regressao_univariada(x, y):
    
    # Gera a REGRESSAO LINEAR UNIVARIADA -> y = b*x + c + e
    X = sm.add_constant(x)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = par_subset[tickers[0]]-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore 

def regressao_multivariada(x, y, periodo):  
    
    # Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore, res


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    print(residuos_shifted)
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    print(res.summary)
    return res.params[1]

# ADF da Statsmodel
def adftest(df,reg):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    ## %ADF clássico, de acordo com o output da função
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    ## %ADF do Ferro
    # Utiliza duas tabelas - uma para quando o tempo é significante, outra quando não é (indep. do noobs)
    ttest = reg.params[1]/reg.bse[1]
    critical_value_tempo = scipy.stats.t.ppf(0.01,df=(len(df)-3)) # Retorna o inverso bicaudal da distribuição t de Student

    if   adf[0] < -4.32:         adfc_2='99%'
    elif adf[0] < -3.67:         adfc_2='95%'
    elif adf[0] < -3.28:         adfc_2='90%'
    else:                        adfc_2='-' 
        
    if   adf[0] < -3.58:         adfc_3='99%'
    elif adf[0] < -3.22:         adfc_3='95%'
    elif adf[0] < -2.60:         adfc_3='90%'
    else:                        adfc_3='-'                 

    
    if (abs(ttest) < critical_value_tempo): aceitar_t0 = '99%'
    else:  aceitar_t0 = '0%'
        
        
    return adf[0], adfc, adf[1], adfc_2, adfc_3, aceitar_t0      

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = x.tail(1)[0]
    ultimo_y = y.tail(1)[0]
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv

def calculo_meia_vida(residuo):
    # Retirado da planilha do Ferro
    coef_ang_residuos = regressao_residuos(residuo) 
    mv_beta = -1*np.log(1+coef_ang_residuos)
    meia_vida = 2/mv_beta  
    return meia_vida

def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return "{} Dias".format(int(round(half_life)))

In [760]:
## Funcoes graficas

def grafico_residuo(residuo):
        
    plt.figure(figsize=(20,4))
    ax = residuo.plot(color='g', grid=True, label='Z-Score')
    plt.axhline(residuo.mean(), color='red')
    plt.axhline(residuo.mean()+2*residuo.std(), color='blue')
    plt.axhline(residuo.mean()-2*residuo.std(), color='blue')

    ax.xaxis_date()  # formata o timestamp para o formato data
    ax.set_axisbelow(True)
    ax.set_title('Residuos', color='black')
    ax.set_facecolor('white')
    ax.figure.set_facecolor('white')

    plt.legend()
    plt.show()    
        

In [866]:

tickers = ['ENGI11','NTCO3'];

# Busca cotacoes no Yahoo Finance
par = Mt5_dados(tickers, '2019-1-1','2020-7-30')

par['ratio']  = par[tickers[0]]/par[tickers[1]]
par['ratio1'] = par[tickers[1]]/par[tickers[0]]


# Cria o cubo de periodos
cubo = pd.DataFrame(columns=['Periodo','Dickey_Fuller','ADF','ADF_0pct','TEMPO','ADF_99pct','Coef_Ang','QTD_Desvios','Fisher_min','Fisher_max'
                             ,'ADF_pvalue','pct_fin','meia_vida'
                             ,#'coint_p', 'coint_dickey','coint_adf'
                            ])

# Popula o cubo de periodos
#for i in range(100,260,10):
for i in [100,120,140,160,180,200,220,240,250]:
 
    # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
    par_subset = par.tail(i)
    x = par_subset[tickers[1]]
    y = par_subset[tickers[0]]
   
    # Adiciona o PERIODO analisado no dataframe
    cubo = cubo.append({'Periodo': int(i)}, ignore_index=True)
    
    # Gera a REGRESSAO e retorna o Coef Ang, os residuos e o z-score. 
    # Retorna tbm o vetor de retorno da regressao para ser usado no ADF
    coef_ang, residuo, zscore, reg  = regressao_multivariada(x, y, i)

    # Teste de Estacionariedade dos Resíduos (ADF)
    adfstat,adfc,adfpvalue,adfc_2,adfc_3,aceitar_t0 = adftest(residuo, reg)
 
    # Calculo do Fisher
    ficher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.diff().fillna(method="bfill"), y.diff().fillna(method="bfill"))
    
    # Calculo do % Financeiro (C/V)
    pct_fin = pct_financeiro(x,y,coef_ang, zscore.tail(1).values)
    
    # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
    meia_vida = calculo_meia_vida1(residuo)
    
    
    # Preenchimento do cubo
    cubo.loc[cubo['Periodo']==i, 'Coef_Ang'     ] = coef_ang 
    cubo.loc[cubo['Periodo']==i, 'QTD_Desvios'  ] = zscore.tail(1)[0]
    cubo.loc[cubo['Periodo']==i, 'Dickey_Fuller'] = adfstat
    cubo.loc[cubo['Periodo']==i, 'ADF'          ] = adfc 
    cubo.loc[cubo['Periodo']==i, 'ADF_pvalue'   ] = adfpvalue 
    cubo.loc[cubo['Periodo']==i, 'Fisher_min'   ] = "{:.0%}".format(fisher_lo)
    cubo.loc[cubo['Periodo']==i, 'Fisher_max'   ] = "{:.0%}".format(fisher_hi)
    cubo.loc[cubo['Periodo']==i, 'pct_fin'      ] = pct_fin
    cubo.loc[cubo['Periodo']==i, 'meia_vida'    ] = meia_vida
    cubo.loc[cubo['Periodo']==i, 'ADF_0pct'     ] = adfc_2
    cubo.loc[cubo['Periodo']==i, 'ADF_99pct'    ] = adfc_3
    cubo.loc[cubo['Periodo']==i, 'TEMPO'        ] = aceitar_t0



    # Coint - apenas para comparacao
    #score, pvalue, _ = coint(residuo[1:],residuo.diff()[1:], maxlag=1, autolag='t-stat')

    #if (score < _[0]): coint_adf='99%'
    #elif (score < _[1]): coint_adf='95%'
    #elif (score < _[2]): coint_adf='90%'
    #else: coint_adf='0%'
    #cubo.loc[cubo['Periodo']==i, 'coint_dickey']    = score
    #cubo.loc[cubo['Periodo']==i, 'coint_p']       = pvalue
    #cubo.loc[cubo['Periodo']==i, 'coint_adf']     = coint_adf
    
#par.to_excel("Par.xlsx")    
display(cubo)

# Calculo do Beta Rotation
# - Selecionando o maior periodo cointegrado
janela_observacao = 50
max_periodo_coint = int(cubo.loc[cubo['ADF_0pct'] != '-'].tail(1)['Periodo'])
janela_grafico = int(max_periodo_coint * 0.2)

par_subset = par.tail(janela_grafico+janela_observacao).copy()
par_subset.reset_index(level=0, inplace=True) # passar o index para uma coluna
par_subset['betas']='-'

#for i in range(janela_observacao, max_periodo_coint,1):
#
#    y = par_subset[tickers[0]].loc[i-janela_observacao:i-1:1]
#    x = par_subset[tickers[1]].loc[i-janela_observacao:i-1:1]
#    par_subset.loc[i, 'betas'] = regressao_multivariada(x,y, 50)[0]

print(par_subset)

    
# Desvio médio do Coef. Angular - apenas dos periodos cointegrados
cubo_cointegrado = cubo.loc[cubo['ADF_0pct'] != '-']
print("Desvio Medio do Coef. Angular: {:.2%}" .format(cubo_cointegrado['Coef_Ang'].std()/cubo_cointegrado['Coef_Ang'].mean()))



# Selecionar o periodo para mostrar o gráfico dos residuos
@ipywidgets.interact(Periodo=(["Selecione o Período", 100,120,140,160,180,200,220,240,250]))
def Grafico(Periodo):
    if (Periodo != "Selecione o Período"):  
        zscore = regressao_multivariada(par[tickers[1]].tail(Periodo), par[tickers[0]].tail(Periodo),Periodo)[2]
        grafico_residuo(zscore)
        print(zscore)
        grafico_residuo(par_subset.loc[50:250]['betas'])
       

,Periodo,Dickey_Fuller,ADF,ADF_0pct,TEMPO,ADF_99pct,Coef_Ang,QTD_Desvios,Fisher_min,Fisher_max,ADF_pvalue,pct_fin,meia_vida
0,100.00,-4.35,99%,99%,0%,99%,0.85,-1.74,51%,74%,0.00,120%,5 Dias
1,120.00,-4.06,99%,95%,0%,99%,0.81,-1.54,54%,75%,0.00,126%,7 Dias
2,140.00,-4.07,99%,95%,0%,99%,0.77,-1.52,54%,73%,0.00,133%,8 Dias
3,160.00,-4.34,99%,99%,0%,99%,0.76,-1.38,53%,72%,0.00,134%,8 Dias
4,180.00,-4.55,99%,99%,0%,99%,0.77,-1.60,52%,70%,0.00,132%,8 Dias
5,200.00,-4.85,99%,99%,0%,99%,0.77,-1.72,52%,69%,0.00,132%,8 Dias
6,220.00,-4.97,99%,99%,0%,99%,0.77,-1.86,52%,68%,0.00,132%,9 Dias
7,240.00,-4.97,99%,99%,0%,99%,0.78,-2.01,51%,67%,0.00,131%,9 Dias
8,250.00,-5.11,99%,99%,0%,99%,0.78,-2.03,51%,67%,0.00,131%,9 Dias


          Date  ENGI11  NTCO3  ratio  ratio1 betas
0   2020-03-09   50.84  36.00   1.41    0.71     -
1   2020-03-10   51.52  39.10   1.32    0.76     -
2   2020-03-11   48.16  35.91   1.34    0.75     -
3   2020-03-12   41.26  28.51   1.45    0.69     -
4   2020-03-13   46.50  33.00   1.41    0.71     -
..         ...     ...    ...    ...     ...   ...
95  2020-07-24   48.00  42.60   1.13    0.89     -
96  2020-07-27   49.50  43.18   1.15    0.87     -
97  2020-07-28   50.02  44.12   1.13    0.88     -
98  2020-07-29   49.38  47.09   1.05    0.95     -
99  2020-07-30   49.71  48.79   1.02    0.98     -

[100 rows x 6 columns]
Desvio Medio do Coef. Angular: 3.68%


interactive(children=(Dropdown(description='Periodo', options=('Selecione o Período', 100, 120, 140, 160, 180,…

Atencao

- Calculo da meia vida: o calculo que peguei da planilha difere com os que encontrei na internet. Nenhum deles batia, fiz um mix dos dois via tentativa e erro e agora está bem proximo.. mas acho que temos que entender melhor

- Fisher: baixei uma formula, nao entendi nada.. tbm seria legal entender

- ADF: estou testando de tudo, o %ADF nao bate nem a pau.. coloquei o ADF2 e o ADF3 que estao na planilha do ferro.. mas ele alterna entre os dois de uma forma q nao entendi, baseada na importancia do coeficiente do tempo na regressao.. e mesmo assim parece q nao bateria.. apesar de que para 'CMIG3.SA','ECOR3.SA' bateu top



In [749]:
mt5.shutdown()


True

In [605]:
# testando algumas configuracoes p encontrar a caralha do inverso da t-student bicaudal 
print(1/scipy.stats.t.cdf(0.02,df=117))
print(scipy.stats.t.cdf(0.02,df=117))

print(scipy.stats.t.ppf(0.01,df=117))
print(1/scipy.stats.t.ppf(0.01,df=117))

print(scipy.stats.t.ppf(0.98,df=117))
print(1/scipy.stats.t.ppf(0.98,df=117))


print(scipy.stats.t.cdf(0.98,df=117))
print(1/scipy.stats.t.cdf(0.98,df=117))

1.968653989094031
0.5079612799099333
-2.3586423635706573
-0.4239727122030229
2.0768980803382875
0.4814872763699213
0.8354458988381355
1.1969655981203713
